Dataset retrieved from: https://zenodo.org/record/1188976

In [ ]:
pip install numpy==1.21.0

In [ ]:
pip install opendatasets

In [ ]:
pip install tqdm

In [ ]:
pip install matplotlib

In [ ]:
pip install librosa

In [ ]:
pip install fastai

In [ ]:
pip install sounddevice

In [1]:
from tqdm import tqdm

import matplotlib.pyplot as plt   # plotting
from matplotlib import image as mpimg

import librosa                    # Python package for music and audio analysis
import librosa.display            # Allows you to display audio files 
import os                         # The OS module in Python provides a way of 
                                  # using operating system dependent functionality.
import scipy.io.wavfile           # Open a WAV files
from scipy.io.wavfile import write

import numpy as np                # Used for working with arrays
import fastai                     
import glob                       # Used to return all file paths that match a specific 
                                  # pattern

import sounddevice as sd

import PIL

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Import fast AI stuff
# from fastai import *                                 
# from fastai.vision.all import *
# from fastai.vision.data import ImageDataLoaders
# from fastai.tabular.all import *
# from fastai.text.all import *
# from fastai.vision.widgets import *

# DOWNLOAD DATASET

In [21]:
# RAVDESS Emotional Speed audio Dataset
import opendatasets as od
od.download("https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio")
# use kaggle credentials

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: moiezqamar
Your Kaggle Key: ········


100%|████████████████████████████████████████| 429M/429M [00:44<00:00, 10.0MB/s]


In [22]:
# RAVDESS Emotional Song audio Dataset
import opendatasets as od
od.download("https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-song-audio")
# use kaggle credentials

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: moiezqamar
Your Kaggle Key: ········


100%|████████████████████████████████████████| 456M/456M [00:46<00:00, 10.4MB/s]


In [36]:
# CREMA-D Dataset
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ejlok1/cremad")
# use kaggle credentials
# information about this dataset: https://github.com/CheyneyComputerScience/CREMA-D

Skipping, found downloaded files in "./cremad" (use force=True to force download)


In [37]:
# emotions
emotion_map = {
    '01' : 'neutral', 
    '02' : 'calm', 
    '03' : 'happy', 
    '04' : 'sad', 
    '05' : 'angry', 
    '06' : 'fearful', 
    '07' : 'disgust', 
    '08' : 'surprised',
    'NEU': '01',
    'HAP': '03',
    'SAD': '04',
    'ANG': '05',
    'FEA': '06',
    'DIS': '07'
}

In [41]:
st = "1022_ITS_ANG_XX.wav"
print(st[9:12])

ANG


In [34]:
# Modify CREMA-D Dataset filenames
directory = './cremad/AudioWAV/'
count = 0
for filename in os.listdir(directory):
    if count == 20:
        break
    print(filename)
    count +=1
    '''
    if filename.startswith("A"):
        new_filename = filename[1:]
        os.rename(filename, new_filename)'''


1022_ITS_ANG_XX.wav
1037_ITS_ANG_XX.wav
1060_ITS_NEU_XX.wav
1075_ITS_NEU_XX.wav
1073_IOM_DIS_XX.wav
1066_IOM_DIS_XX.wav
1078_IWL_SAD_XX.wav
1029_TAI_FEA_XX.wav
1039_IEO_SAD_MD.wav
1008_TAI_HAP_XX.wav
1018_TSI_FEA_XX.wav
1028_IEO_SAD_HI.wav
1039_TSI_HAP_XX.wav
1064_ITS_HAP_XX.wav
1071_ITS_HAP_XX.wav
1045_ITS_FEA_XX.wav
1050_ITS_FEA_XX.wav
1018_IWL_ANG_XX.wav
1059_IEO_ANG_MD.wav
1054_MTI_DIS_XX.wav


# CREATE NEEDED DIRECTORIES

In [23]:
!rm -r "./ravdess-emotional-speech-audio/audio_speech_actors_01-24"
!rm -r "./ravdess-emotional-song-audio/audio_song_actors_01-24"


!mkdir './output'
!mkdir './output/live_images'

!mkdir './Categorical_Dataset'          
!mkdir './Categorical_Dataset/positive' # happy, surprised
!mkdir './Categorical_Dataset/negative' # angry, sad, disgusted, fearful
!mkdir './Categorical_Dataset/neutral'  # neutral, calm

# !mkdir './DATASET'
# !mkdir './DATASET/angry'
# !mkdir './DATASET/calm'
# !mkdir './DATASET/disgust'
# !mkdir './DATASET/fearful'
# !mkdir './DATASET/neutral'
# !mkdir './DATASET/sad'
# !mkdir './DATASET/happy'
# !mkdir './DATASET/surprised'

!mkdir './models'

mkdir: ./output: File exists
mkdir: ./output/live_images: File exists
mkdir: ./models: File exists


# DEFINING THE PATHS

In [26]:
AUDIO_FOLDER1 = "./ravdess-emotional-speech-audio/*"
AUDIO_FOLDER2 = "./ravdess-emotional-song-audio/*"


DATASET = "./Categorical_Dataset/"


paths = [audioFile for actor in glob.glob(AUDIO_FOLDER1) for audioFile in glob.glob(actor +'/*')] + [audioFile for actor in glob.glob(AUDIO_FOLDER2) for audioFile in glob.glob(actor +'/*')]

result = [paths[i:i+10] for i in range(0, len(paths), 10)]


# RUN THIS CODE TO CLEAR THE DATASET FOLDER:

In [29]:
# clear the test1 and train1 folders:
directories = [DATASET] # specify the path to the directory

for directory in directories:
    for foldername in os.listdir(directory):
        folderpath = os.path.join(directory, foldername)
        if os.path.isdir(folderpath):
            for filename in os.listdir(folderpath):
                filepath = os.path.join(folderpath, filename)
                if os.path.isfile(filepath):
                    os.remove(filepath)

# CONVERT ALL AUDIO FILES TO MELSPECTROGRAM:

In [30]:
def convert_audio_to_melspectrogram(audioPath, savePath):

    # Load audio file and visualize its waveform (using librosa)
    # The `librosa.load()` function takes two arguments: the path to the audio file and the sample rate.
    # The sample rate is the number of samples per second in the audio file.
    x, sr = librosa.load(audioPath, sr=44100)

    # Trim the audio file to remove silence at the beginning and end.
    # The `librosa.effects.trim()` function takes two arguments: the audio signal and the threshold.
    # The threshold is the minimum amplitude that is considered to be non-silence.
    xt,_=librosa.effects.trim(x)                         

    x=xt

    # Plot the spectrogram.
    # The `librosa.stft()` function computes the short-time Fourier transform of the audio signal.
    # The `librosa.amplitude_to_db()` function converts the amplitude of the spectrogram to decibels.
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))

    # Apply log transformation on the loaded audio signals
    # The `librosa.display.specshow()` function plots the spectrogram.
    # The `sr` argument specifies the sample rate of the audio signal.
    # The `x_axis` argument specifies the axis along which the time is displayed.
    # The `y_axis` argument specifies the axis along which the frequency is displayed.
    librosa.display.specshow(Xdb, sr=sr, vmin=-60, vmax=60,x_axis='time', y_axis='log',cmap='magma', ax=None)

    #plt.colorbar()
    
    # Remove the axis
    plt.gca().get_xaxis().set_visible(False)
    plt.gca().get_yaxis().set_visible(False)
    
    # Save the figure.
    # The `plt.savefig()` function saves the figure to a file.
    # The `savePath` argument specifies the path to the file.
    plt.savefig(savePath)
    plt.clf()

In [31]:
# convert_audio_to_melspectrogram(result[0][0],"./output/sup")
# convert_audio_to_melspectrogram(result[0][1],"./output/sup")
# convert_audio_to_melspectrogram(result[0][2],"./output/sup")

In [32]:
# Convert the audio files to melspectrograms.
counts = {}
fileLoc=1
for path in result:
    files = tqdm(path)
    for audio_file_path in files:
        files.set_postfix_str(f"{fileLoc}/{len(result)}")
        
        em = audio_file_path[-18:-16]
        
        emotion = ''
        
        if em in ('01','02'):
            emotion =  'neutral'
        elif em in ('03','08'):
            emotion = 'positive'
        elif em in ('04','05','06','07'):
            emotion = 'negative'
        else:
            em = emotion_map[audio_file_path[9:12]]
            if em in ('01','02'):
                emotion =  'neutral'
            elif em in ('03','08'):
                emotion = 'positive'
            elif em in ('04','05','06','07'):
                emotion = 'negative'
            
        
        count = counts.get(emotion, 1)
                
        p = os.path.join(f"{DATASET}{emotion}", f"{emotion}{str(count).zfill(5)}_{dicts[em]}.jpg")        
        
        counts[emotion] = count + 1
        
        convert_audio_to_melspectrogram(audio_file_path, p)
    fileLoc+=1

100%|██████████████████████████████████| 10/10 [00:02<00:00,  3.34it/s, 101/246]


  0%|                                           | 0/10 [00:00<?, ?it/s, 138/246]


KeyboardInterrupt: 

IndexError: list index out of range

<Figure size 640x480 with 1 Axes>

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

fs=44100
seconds=3
dtype = 'int16'
mic = int(input(f'Of the following list of devices, select which one to use as your microphone:\n{sd.query_devices()}\n>>>'))
count = 1

while true:
    
    print("Recording...")
    
    checkEmotion(count,mic, fs, seconds, dtype)
    
    cont = input("\nContinue? (Y/N): ")
    
    if cont == 'Y' or cont == 'y':
        count+=1
        continue
    else:
        break

In [ ]:
'''loss_functions = [CrossEntropyLossFlat(), LabelSmoothingCrossEntropy(),
                  FocalLossFlat()]
loss_functions1 = ["CrossEntropyLossFlat", "LabelSmoothingCrossEntropy",
                  "FocalLossFlat"]
train_size = [0.2, 0.4, 0.6]

for LF in range(0,3):
    for TS in range(0,3):
        print(f"___________{loss_functions1[LF]} ... {train_size[TS]}___________")
        dls = ImageDataLoaders.from_folder(train_path, valid_pct=train_size[TS], seed=5, num_workers=0)
        print(f"validation: {len(dls.valid_ds.items[:])}  |  training set: {len(dls.train_ds.items[:])}")

        learn = vision_learner(dls, models.resnet34, loss_func=loss_functions[LF], metrics=accuracy)
        lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
        print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

        learn.fit(10, float(f"{lr_steep:.2e}"))
        #interp = ClassificationInterpretation.from_learner(learn)
        #losses,idxs = interp.top_losses()
        #len(dls.valid_ds)==len(losses)==len(idxs)

        #interp.plot_confusion_matrix(figsize=(8,8), dpi=90)

        #interp.plot_top_losses(2, figsize=(10,11))
        
        print("\n")
'''

In [ ]:
import pathlib
data_dir = pathlib.Path('./DATASET').with_suffix('')

In [ ]:
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
anger = list(data_dir.glob('angry/*'))
PIL.Image.open(str(anger[0]))
PIL.Image.open(str(anger[1]))

calm = list(data_dir.glob('calm/*'))
PIL.Image.open(str(calm[0]))
PIL.Image.open(str(calm[1]))

In [ ]:
batch_size = 32
img_height = 240
img_width = 320

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Get the predictions from the model
predictions = model.predict(val_ds)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels
true_labels = []
for images, labels in val_ds:
    true_labels.extend(labels.numpy())

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Fight overfitting by using data augmentation
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")
        


In [ ]:
# Utilizing Dropout
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Get the predictions from the model
predictions = model.predict(val_ds)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels
true_labels = []
for images, labels in val_ds:
    true_labels.extend(labels.numpy())

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
location = './DATASET/angry/angry000001.jpg'
#file_path = tf.keras.utils.get_file('thingo', origin=location)

#img = pathlib.Path('./DATASET/angry/angry000001.jpg')

img = tf.keras.utils.load_img(
   location, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])



sorted_predictions = np.argsort(score)[::-1]

# Take the top five predictions.
top_predictions = sorted_predictions[:5]

# Print the top five predictions.
for i in range(len(top_predictions)):
    print(
        "The image most likely belongs to {} with a {:.4f} percent confidence."
        .format(class_names[top_predictions[i]], 100 * score[top_predictions[i]])
    )